In [7]:
!pip install pulp
!pip install deap
!pip install simanneal

In [8]:
from pulp import LpProblem, LpMaximize, LpVariable
from deap import base, creator, tools, algorithms
from simanneal import Annealer
import random

def knapsack_exact(weights, values, capacity):
    num_items = len(weights)
    knapsack_model = LpProblem("Knapsack Problem", LpMaximize)

    # Decision Variables
    x = [LpVariable(f"x{i}", lowBound=0, upBound=1, cat='Integer') for i in range(num_items)]

    # Objective Function
    knapsack_model += sum(x[i] * values[i] for i in range(num_items))

    # Constraint: Total weight and size
    knapsack_model += sum(x[i] * weights[i] for i in range(num_items)) <= capacity

    # Solve the problem
    knapsack_model.solve()

    selected_items = [i for i in range(num_items) if x[i].value() == 1]
    return knapsack_model.objective.value(), selected_items


def knapsack_genetic(weights, values, capacity):
    def evaluate(individual):
        total_weight = sum(weights[i] for i in range(len(individual)) if individual[i] == 1)
        total_value = sum(values[i] for i in range(len(individual)) if individual[i] == 1)
        return (total_value,) if total_weight <= capacity else (0,)

    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)

    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=len(weights))
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", evaluate)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)

    population = toolbox.population(n=100)
    algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=50, verbose=False)

    best_individual = tools.selBest(population, k=1)[0]
    selected_items = [i for i in range(len(best_individual)) if best_individual[i] == 1]
    total_value = sum(values[i] for i in selected_items)

    return total_value, selected_items


class KnapsackAnnealer(Annealer):
    def __init__(self, state, weights, values, capacity):
        self.weights = weights
        self.values = values
        self.capacity = capacity
        super().__init__(state)  # state is a list of 0s and 1s representing item selection

    def move(self):
        idx = random.randint(0, len(self.state) - 1)
        self.state[idx] ^= 1  # Flip 0 to 1 or 1 to 0

    def energy(self):
        total_weight = sum(self.weights[i] for i in range(len(self.state)) if self.state[i] == 1)
        total_value = sum(self.values[i] for i in range(len(self.state)) if self.state[i] == 1)
        return total_value if total_weight <= self.capacity else 0

def knapsack_simulated_annealing(weights, values, capacity):
    initial_state = [random.randint(0, 1) for _ in range(len(weights))]
    annealer = KnapsackAnnealer(initial_state, weights, values, capacity)
    state, energy = annealer.anneal()

    selected_items = [i for i in range(len(state)) if state[i] == 1]
    return energy, selected_items


def knapsack_constructive_heuristic(weights, values, capacity):
    num_items = len(weights)
    selected_items = []
    total_weight = 0
    total_value = 0

    # Greedily select items based on value-to-weight ratio
    value_weight_ratio = [(values[i] / weights[i], i) for i in range(num_items)]
    value_weight_ratio.sort(reverse=True)

    for ratio, index in value_weight_ratio:
        if total_weight + weights[index] <= capacity:
            selected_items.append(index)
            total_weight += weights[index]
            total_value += values[index]

    return total_value, selected_items


In [9]:
num_items = 20
weights = [random.randint(1, 10) for _ in range(num_items)]
values = [random.randint(1, 100) for _ in range(num_items)]
capacity = 50

# Exact Optimization
exact_solution = knapsack_exact(weights, values, capacity)

# Genetic Algorithm
genetic_solution = knapsack_genetic(weights, values, capacity)

# Simulated Annealing
simulated_annealing_solution = knapsack_simulated_annealing(weights, values, capacity)

# Constructive Heuristic Approach
heuristic_solution = knapsack_constructive_heuristic(weights, values, capacity)

print("Exact Optimization Solution:", exact_solution)
print("Genetic Algorithm Solution:", genetic_solution)
print("Simulated Annealing Solution:", simulated_annealing_solution)
print("Heuristic Approach Solution:", heuristic_solution)

/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
 Temperature        Energy    Accept   Improve     Elapsed   Remaining


Exact Optimization Solution: (709.0, [0, 4, 5, 7, 10, 11, 12, 13, 14, 18])
Genetic Algorithm Solution: (708, [0, 3, 5, 7, 10, 11, 12, 13, 14, 15, 18])
Simulated Annealing Solution: (0, [1, 2, 3, 5, 6, 7, 8, 10, 11, 15, 16, 18])
Heuristic Approach Solution: (708, [5, 14, 12, 0, 10, 13, 7, 11, 18, 15, 3])
